In [4]:
from census import Census
from us import states
import us
import requests
import zipfile
import io
import geopandas as gpd
import pandas as pd
import os
import socket
from timeit import default_timer as timer

In [5]:
country='US'

key = '5984786462aaccc39f904bf750cc8274bd40f9f8'

if os.getenv('CLUSTER')=='PRINCE':
    path_to_data='/scratch/spf248/covid/data'
else:
    path_to_data='../data'

dict_of_states = {'AK':['02','ALASKA'], 'MS':['28','MISSISSIPPI'],'AL':['01','ALABAMA'], 'MT':['30','MONTANA'],
'AR':['05','ARKANSAS'],'NC':['37','NORTH CAROLINA'], 'AS':['60','AMERICAN SAMOA'],
 'ND':['38','NORTH DAKOTA'], 'AZ':['04','ARIZONA'], 'NE': ['31','NEBRASKA'],
'CA':['06','CALIFORNIA'], 'NH':['33','NEW HAMPSHIRE'], 'CO':['08', 'COLORADO'], 'NJ':['34', 'NEW JERSEY'],
'CT':['09', 'CONNECTICUT'], 'NM':['35', 'NEW MEXICO'],'DC':['11','DISTRICT OF COLUMBIA'], 'NV':['32','NEVADA'],
'DE':['10', 'DELAWARE'], 'NY':['36','NEW YORK'], 'FL':['12','FLORIDA'], 'OH':['39','OHIO'],
'GA':['13','GEORGIA'], 'OK':['40','OKLAHOMA'], 'GU':['66','GUAM'], 'OR':['41','OREGON'],
'HI':['15','HAWAII'],'PA':['42','PENNSYLVANIA'],'IA':['19','IOWA'], 'PR':['72','PUERTO RICO'],
'ID':['16','IDAHO'],'RI':['44','RHODE ISLAND'],'IL':['17','ILLINOIS'],'SC':['45','SOUTH CAROLINA'],
'IN':['18','INDIANA'],'SD':['46','SOUTH DAKOTA'],'KS':['20','KANSAS'],'TN':['47','TENNESSEE'],
'KY':['21','KENTUCKY'],'TX':['48','TEXAS'],'LA':['22','LOUISIANA'],'UT':['49','UTAH'],'MA':['25','MASSACHUSETTS'],
'VA':['51','VIRGINIA'],'MD':['24','MARYLAND'],'VI':['78','VIRGIN ISLANDS'],'ME':['23','MAINE'],
'VT':['50','VERMONT'],'MI':['26','MICHIGAN'],'WA':['53','WASHINGTON'],'MN':['27','MINNESOTA'],'WI':['55','WISCONSIN'],
'MO':['29','MISSOURI'],'WV':['54','WEST VIRGINIA'],'WY': ['56','WYOMING']}

In [ ]:
def download_shp_and_save():
    '''
    Creates folders for each state, downloads the shapefile, unzip it and saves it into a folder
    for each state
    '''
    if not os.path.exists(os.path.join(path_to_data,'shapefiles',country)):
            os.mkdir(os.path.join(path_to_data,'shapefiles',country))
        
    for state, state_info in dict_of_states.items():
        
        if not os.path.exists(os.path.join(path_to_data,'shapefiles',country,state_info[1])):
            
            os.mkdir(os.path.join(path_to_data,'shapefiles',country,state_info[1]))
            
            link = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_{}_tabblock10.zip'.format(state_info[0])

            print(state_info[1])
            print(link)

            req = requests.get(link)
            file = zipfile.ZipFile(io.BytesIO(req.content))
            file.extractall(path= os.path.join(path_to_data,'shapefiles',country,state_info[1]))
        
        else:
            
            print(state_info[1], 'already downloaded')

In [ ]:
print('Download US Census')
start = timer()

download_shp_and_save()

print("Done in", round(timer()-start), "sec")

In [ ]:
def open_shape(state):
    '''
    Opens a shapefile as a geopandas object
    inputs:
        state(str): valid values are State FIP codes ("AK", "CA", etc)
    returns:
        geopandas dataframe
    '''
    rv = gpd.read_file(path_to_data + '/shapefiles/' + country + '/' + dict_of_states[state][1] + '/tl_2010_{}_tabblock10.shp'.format(dict_of_states[state][0]))
    return rv

In [ ]:
print('Create csvs for Geospark')
start = timer()

for i,state in enumerate(dict_of_states):
    if not os.path.exists(os.path.join(path_to_data,'shapefiles',country,'polygons',state+'.csv')):
        print(state)
        open_shape(state)[['GEOID10','geometry']].to_csv(
        os.path.join(path_to_data,'shapefiles',country,'polygons',state+'.csv'),index=False)
    
print("Done in", round(timer()-start), "sec")